In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.self_supervised import *

Loading imports


In [2]:
torch.cuda.set_device(0)

In [3]:
dls = get_ss_data(256, sz=128, splits=Meta.splits_stg1)

In [4]:
lclass = partial(loss_class, f=get_loss())
lcont = partial(loss_cont, f=BatchContrastiveLoss())

ccb = CombinedContrastiveCallback(size=128)
learn = cnn_learner(dls, xresnet18, loss_func=CombinedContrastiveLoss(), lr=3e-3,
            opt_func=Adam, metrics=[accuracy_ss, lcont, lclass], cbs=ccb)

learn.model = CombinedModel(learn.model)

In [5]:
name = 'self_supervised_train_1'

In [ ]:
do_fit(learn, 12, 4e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_ss,loss_cont,loss_class,time
0,0.166666,0.233227,0.916545,0.093838,0.139389,25:49
1,0.154233,0.217061,0.923267,0.085691,0.131370,25:41
2,0.148429,0.256627,0.922452,0.070249,0.186378,26:01
3,0.138063,0.202106,0.919129,0.061819,0.140286,26:22


In [ ]:
learn.load(f'runs/{name}-1')
learn.dls = get_ss_data(128, sz=256, splits=Meta.splits_stg1)
ccb.update_size(256)
do_fit(learn, 12, 4e-3)
learn.save(f'runs/{name}-2')

In [ ]:
learn.load(f'runs/{name}-2')
learn.dls = get_ss_data(64, sz=384, splits=Meta.splits_stg1, img_dir=path_jpg)
ccb.update_size(384)
do_fit(learn, 4, 4e-4)
learn.save(f'runs/{name}-3')